In [301]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

<h3>Podemos prever se a pessoa está armada ou não, utilizando apenas dados demográficos?</h3> 
<p>Não, não podemos, como foi verificado a partir de diversas versões do classificador abaixo. O classificador errava bastante pois não parecia haver correlação suficiente entre os dados demográficos e a pessoa estar armada.</p>
<h3> Mas podemos prever razoavelmente bem se a pessoa está armada ou não utilizando também os dados sobre tentativa de fuga, nível de ameaça e sinais de doença mental! </h3>

In [290]:
df = pd.read_csv('./datasets/shootings.csv')
df.head()

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,arms_category
0,3,Tim Elliot,2015-01-02,shot,gun,53.0,M,Asian,Shelton,WA,True,attack,Not fleeing,False,Guns
1,4,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,White,Aloha,OR,False,attack,Not fleeing,False,Guns
2,5,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.0,M,Hispanic,Wichita,KS,False,other,Not fleeing,False,Unarmed
3,8,Matthew Hoffman,2015-01-04,shot,toy weapon,32.0,M,White,San Francisco,CA,True,attack,Not fleeing,False,Other unusual objects
4,9,Michael Rodriguez,2015-01-04,shot,nail gun,39.0,M,Hispanic,Evans,CO,False,attack,Not fleeing,False,Piercing objects


Vamos binarizar alguns dados, como <b>pessoa armada</b> e <b>gênero</b>

In [291]:
df['armedBinary'] = df['armed'] != 'unarmed'
df['armedBinary'] = df['armedBinary'].astype(int)
df = df.drop('armed', axis= 1)

df['genderBinary'] = df['gender'] == 'M'
df['genderBinary'] = df['genderBinary'].astype(int)
df = df.drop('gender', axis=1)

Dropamos alguns dados que não faziam diferença significativa no resultado e fizemos <b>One-hot-encoding</b> com dados categóricos.

In [292]:
df = df.drop(['id', 'name', 'date', 'manner_of_death', 'city',
             'body_camera', 'arms_category'], axis=1)
df

df = pd.get_dummies(df, columns=['race', 'state', 'flee', 'threat_level', 'signs_of_mental_illness'])

Separamos os dados de <b>teste e treino</b>.

In [293]:
X = df.copy()
y = df['armedBinary']

X = X.drop('armedBinary', axis=1)

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=10)

Podemos ver que as <b>classes</b> que queremos predizer estão bastante <b>desbalanceadas</b> nesse dataset:

In [294]:
Y_train.value_counts()

1    3627
0     289
Name: armedBinary, dtype: int64

Para corrigir essa distorção, resolvemos fazer um <b>upsample</b> dos dados da classe que aparece menos.<br>
Para isso juntamos os dados de treino, alteramos a proporção de dados de cada classe para 1/2 e depois separamos os dados de treino novamente.

In [295]:
train_data = pd.concat([X_train, Y_train], axis=1)

positive = df[df.armedBinary==1]
negative = df[df.armedBinary==0]

negative_upsampled = resample(negative, 
                                 replace=True,
                                 n_samples=len(positive),
                                 random_state=1)
newDf = pd.concat([positive, negative_upsampled]).reset_index(drop=True)

In [296]:
print(newDf['armedBinary'].value_counts())
Y_train = newDf['armedBinary']
X_train = newDf.drop('armedBinary', axis=1)

1    4547
0    4547
Name: armedBinary, dtype: int64


Agora podemos <b>normalizar os dados</b> de treino e teste.

In [297]:
x_train_mean = X_train.mean()
x_train_std = X_train.std(ddof=1)

newX_train = X_train.copy()
newX_train = newX_train - x_train_mean
newX_train = newX_train / x_train_std

newX_test = X_test.copy()
newX_test = newX_test - x_train_mean
newX_test = newX_test / x_train_std

Utilizamos o <b>GridSearchCV</b> para estimar o <b>melhor número de vizinhos</b>.

In [298]:
model = KNeighborsClassifier()

params = {"n_neighbors": np.arange(1, 10, 2)}
clf = GridSearchCV(model, params, cv=5)
clf.fit(newX_train, Y_train)
k = clf.best_params_['n_neighbors']
k

3

<b>Treinamos o modelo</b> com esse número.

In [299]:
model = KNeighborsClassifier(n_neighbors=k)
model.fit(newX_train, Y_train)

KNeighborsClassifier(n_neighbors=3)

Agora <b>validamos</b> com os <b>dados de teste</b>

In [309]:
predicted = model.predict(newX_test)
predicted

print("Acurácia: ", accuracy_score(Y_test, predicted))
print("Outras métricas: \n")
print(classification_report(Y_test, predicted))

Acurácia:  0.9427987742594485
Outras métricas: 

              precision    recall  f1-score   support

           0       0.51      0.95      0.67        59
           1       1.00      0.94      0.97       920

    accuracy                           0.94       979
   macro avg       0.76      0.95      0.82       979
weighted avg       0.97      0.94      0.95       979

